# HW01
* 이름: 아무개
* 학번: 99990000

위에 이름과 학번을 자신의 것으로 기입하고 또 맨 위에 노트북의 제목도 `HW01<이름><학번>`으로 바꾸어 HiClass를 통해 마감시간까지 제출하면 됩니다.
마감시간은 HiClass에 공지되어 있습니다.

In [3]:
class RGB {
    int r, g, b;
    RGB() { r=0; g=0; b=0; }
    RGB(int _r, int _g, int _b) { r=_r; g=_g; b=_b; }
    @Override public String toString() { return "rgb("+r+","+g+","+b+")"; }
}

com.twosigma.beaker.javash.bkr5a0ab621.RGB

In [4]:
class Point {
    int x, y;
    Point() { x=0; y=0; }
    Point(int _x, int _y) { x=_x; y=_y; }
    @Override public String toString() { return "("+x+","+y+")"; }
}

com.twosigma.beaker.javash.bkr5a0ab621.Point

In [5]:
// 연산용 
class Calc{

    static double Length(Point p1, Point p2) //두 점 사이의 거리 - 원,삼각형 연산시 사용
    {
        double x = p1.x - p2.x; 
        double y = p1.y - p2.y;
        return Math.sqrt(x*x+y*y);
    }
    static int Area(double lineA,double lineB,double lineC) // 넓이, 삼각형 연산시 사용
    {
        double length = (lineA + lineB + lineC) / 2;
        double area = Math.sqrt(length*(length-lineA)*(length-lineB)*(length-lineC));
        return (int)Math.round(area);
    }
    static boolean EqArea(Point a,Point b,Point c,Point p) // 넓이가 같은가? , 삼각형 연산시 사
    {
        double lineab   = Length(a,b);
        double linebc   = Length(b,c);
        double lineac   = Length(a,c);
        double lineap   = Length(a,p);
        double linebp   = Length(b,p);
        double linecp   = Length(c,p);
        int areaabp = Area(lineap,linebp,lineab);
        int areabcp = Area(linecp,linebp,linebc);
        int areaacp = Area(lineap,linecp,lineac); 
        int areaabc = Area(lineab,linebc,lineac);
        if(areaabc == (areaabp+areabcp+areaacp)) // 내부에 점이 있다면 해당 점으로 만든 삼각형 3개의 넓이가 원래 삼각형과 같을것이므로
            return true;
        else
            return false;
    }
    static Point Shortlen(int x, int y,double px,double py, Point a,Point c,Point p) // 선분 ac와 점 p의 가장 가까운 접점은 어디인가?
    {
            double min = 9999999; // 초기값이므로 크게
            double minx=0,miny=0;
            double length = Math.sqrt(x*x+y*y);
            double xr = x/length; // 대각선 점들의 위치변화값
            double yr = y/length;
            double ix,iy,il;
            while((px >= Math.min(a.x,c.x) && px<= Math.max(a.x,c.x)) && (py >= Math.min(a.y,c.y) && py<=Math.max(a.y,c.y)) )
            { // px,py가 대각선의 시작/도착점 범위 안에서만 돌도록
                px += xr; //px,py : 현재 검사하고 있는 점. 초기값은 대각선 변의 끝점중 하나
                py += yr; 
                ix=px-p.x; // 검사할 점과 현재 대각선 위의 점의 좌표차이 
                iy=py-p.y;
                il = Math.sqrt(ix*ix+iy*iy); // 거리
                if(il<min){ // 현재 위치가 더 가깝다면 거리와 좌표를 기
                    min = il;
                    minx = px;
                    miny = py;
                }
            }
            Point mins = new Point((int)minx,(int)miny); // 반복문의 결과인 거리가 가장 짧은 점을 리턴
            return mins;
    }
}

com.twosigma.beaker.javash.bkr5a0ab621.Calc

## 첫번째 설계
* `CircObj`, `RectObj`, `RTriObj` 클래스 작성 (3점)
* 같은 클래스의 인스턴스끼리 (`CircObj`끼리, `RectObj`끼리, `RTriObj`) 충돌 검사 테스트 사례 작성 및 각 테스트 사례를 svg 출력으로 확인 (3점)
* 서로 다른 클래스의 (`CircObj`와 `RectObj`, `RectObj`와 `RTriObj`, `RTriObj`와 `CircObj`) 인스턴스 충돌 검사 테스트 사례 작성 및 각 테스트 사례를 svg 출력으로 확인 (3점)

In [6]:
abstract class ShapeObj { // 어떤 추상적인 모양의 물건 클래스
    Point p;   // 물건 위치에 대한 정보
    RGB color; // 물건의 색깔
    ShapeObj(Point p, RGB c) { this.p=p; color=c; }
    abstract Point closest(Point p); // 점 p와 가장 까까운 도형에 포함된 점 (소수점 이하는 적절히 처리해 정수 좌표로) //원 내부는 자기자신
    abstract boolean collides(Point p); // 점 p와 충돌하는가?
    abstract boolean collides(ShapeObj o); // o와 충돌하는가?
    abstract String toSVG(); // svg 태그 생성
    @Override
    public String toString() { return toSVG(); }
}

com.twosigma.beaker.javash.bkr5a0ab621.ShapeObj

<pre>
원의 경우 어느 방향이던 항상 경계와의 거리는 반지름이므로, 이를 이용하여 충돌 여부를 쉽게 구할수 있다.

우선 점과의 거리가 반지름보다 작다면, 점은 원 안쪽에 있으므로 원과 충돌한 것이다.

그러나 점과의 거리가 반지름보다 크다면, 점은 원 밖에 있으므로 원과 충돌하지 않았을 것이다.

closest의 경우 가까운 점의 좌표를 반환해야 하는데, 아래 그림과 같은 삼각형의 닮음을 이용하여 충돌 위치를 구할수 있다.

<img src="https://github.com/sbj1229/oop2019Spring/raw/master/circle.png"/> </div>


In [105]:
class CircObj extends ShapeObj { // 원 모양 물건 클래스
    // 원의 경우 p는 중심이다
    int r; // 반지름
    CircObj(int r, Point p, RGB c) 
    {
        super(p, c); // 부모 클래스 생성자 호출
        this.r = r;
    }
    // 아래를 완성하라 (1점)
    @Override
    Point closest(Point p) // 점 p와 가장 까까운 도형에 포함된 점 
    { 
        double d = Calc.Length(this.p,p); // 두 점의 거리를 구한다.
        if(r > d) // 반지름이 길이보다 큰 경우, 점이 원 내부에 존재하므로 해당 점을 리턴
            return p; 
        else        // 반지름이 길이보다 작은경우, 직긱삼각형 비례를 이용하여 x,y 좌표를 구하여 리턴한다.
        {
            int dx = (int)Math.round( (p.x-this.p.x)*r/d ); 
            int dy = (int)Math.round( (p.y-this.p.y)*r/d );
            return new Point(this.p.x+dx, this.p.y+dy);
        }
    }
    @Override
    boolean collides(Point p) 
    { 
        double len = Calc.Length(this.p,p); // 두 점의 길이를 구한다.
        if( len <= r ) // 반지름이 길이보다 크거나 같은 경우, 점이 원 내부, 혹은 경계에 존재하므로 충돌
            return true; 
        else           
            return false;
    }
    @Override
    boolean collides(ShapeObj o) 
    { 
        Point po = o.closest(this.p); // 상대 도형과 내 중심의 충돌정보를 저장할 포인트 변수 생성
        if(collides(po)) // po와 내 도형이 충돌한다면 상대 도형과 충돌
            return true;
        else
            return false;
  
    }
    @Override
    String toSVG() 
    {
        return "<circle cx='"+p.x+"' cy='"+p.y+"' r='"+r+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }
}

com.twosigma.beaker.javash.bkr5a0ab621.CircObj

<pre>
직사각형은 가로/세로가 각각 x,y축과 평행하다는 제약조건이 있으므로 대각선/곡선에 따른 추가 연산 필요 없이 단순 비교 연산으로 충돌 여부를 구할수 있다.

점의 x,y를 px,py라 하고 사각형의 점을 rx,ry라 칭하면,
모서리 반환: 
첫번째 case : px가 rx보다 작고, py가 ry보다 작다면  사각형의 x,y     값을 반환 (사각형의 　왼쪽 위에 존재)
두번째 case : px가 rx보다 작고, py가 ry보다 크다면  사각형의 x,y+h   값을 반환 (사각형의 　왼쪽 아래에 존재)
세번째 case : px가 rx보다 크고, py가 ry보다 작다면  사각형의 x+w,y   값을 반환 (사각형의 오른쪽 위에 존재)
네번째 case : px가 rx보다 크고, py가 ry보다 크다면  사각형의 x+w,y+h 값을 반환 (사각형의 오른쪽 아래에 존재)

변의 좌표 반환: 
첫번째 case : px가 rx~rx+w 사이이고, py가 ry보다 작다면  사각형의 x,y    값을 반환 (사각형의 　위쪽에 존재)
두번째 case : px가 rx~rx+w 사이이고, py가 ry보다 크다면  사각형의 x,y+h  값을 반환 (사각형의 아래쪽에 존재)
세번째 case : px가 rx보다 작고, py가  ry~ry+h 사이라면  사각형의 x+w,y   값을 반환 (사각형의 　왼쪽에 존재)
네번째 case : px가 rx보다 크고, py가  ry~ry+h 사이라면  사각형의 x+w,y+h 값을 반환 (사각형의 오른쪽에 존재)

In [109]:
class RectObj extends ShapeObj { // 직사각형 모양 물건 클래스
    // 직사각형의 경우 경우 p는 왼쪽 위 꼭지점이다
    int w; // 너비 (양수)
    int h; // 높이 (양수)

    RectObj(int w, int h, Point p, RGB c) 
    {
        super(p, c); // 부모 클래스 생성자 호출
        this.w = w;
        this.h = h;
    }
    @Override
    Point closest(Point p) 
    { // 점 p와 가장 까까운 도형에 포함된 점 (소수점 이하는 적절히 처리해 정수 좌표로)
        //case 8
        if(p.x < this.p.x && p.y < this.p.y) // 왼쪽 위 모서리
            return new Point(this.p.x,this.p.y);
        else if(p.x < this.p.x && p.y > this.p.y + this.h) // 왼쪽 아래 모서리
            return new Point(this.p.x,this.p.y+this.h);
        else if(p.x > this.p.x + this.w && p.y > this.p.y + this.h) // 오른쪽 아래 모서리
            return new Point(this.p.x+this.w,this.p.y+this.h);
        else if(p.x > this.p.x + this.w && p.y < this.p.y ) // 오른쪽 위 모서리
            return new Point(this.p.x + this.w, this.p.y);    
        else if(p.x < this.p.x && p.y >= this.p.y && p.y <= this.p.y + this.h) // 왼쪽 바깥
            return new Point(this.p.x,p.y);
        else if(p.x >= this.p.x && p.x <= this.p.x + this.w && p.y > this.p.y + this.h) // 아래쪽 바깥
            return new Point(p.x,this.p.y+this.h);
        else if( p.x > this.p.x + this.w && p.y >= this.p.y && p.y <= this.p.y + this.h) // 오른쪽 바깥
            return new Point(this.p.x+this.w,p.y);
        else if(p.x >= this.p.x && p.x <= this.p.x + this.w && p.y < this.p.y) // 위쪽 바깥
            return new Point(p.x,this.p.y);
        else // 모두 아니면 좌표 안에 있으므로 해당 점 리턴 
            return p;
    }
    @Override
    boolean collides(Point p) 
    { 
        Point p2 = closest(p);
        if(p2.x ==p.x && p2.y == p.y) //인자 p와 리턴값이 같다면 도형에 포함, 충돌
            return true;
        else
            return false;
    }
    @Override
    boolean collides(ShapeObj o) 
    { 
        Point po = closest(o.p); // 상대 중심과 내 도형의 위치(충돌-o.p,비충돌-내 도형의 경계) 저장 
        if(o.collides(po)==true) // 내 도형의 중심이나 상대 도형의 경계가 내 도형과 충돌한다면 츙돌
            return true;
        else if (o.collides(this.p) ||   // if의 조건은 대각선/곡선의 경우 실수->정수 변환 과정에서 에러 가능, 따라서 각 모서리 충돌 검사
                 o.collides(new Point(this.p.x+w,this.p.y  )) ||
                 o.collides(new Point(this.p.x,  this.p.y+h)) ||
                 o.collides(new Point(this.p.x+w,this.p.y+h)) ) 
            return true;
        else
            return false; 
    }
    @Override
    String toSVG() 
    {
        return "<rect x='"+p.x+"' y='"+p.y+"' width='"+w+"'height='"+h+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }
}

com.twosigma.beaker.javash.bkr5a0ab621.RectObj

<pre>
직각 삼각형의 경우 x,y축과 평행한 방향의 바깥쪽은 직사각형과 똑같이 처리할 수 있으나, 직각삼각형의 방향은 4종류이므로 각각을 검출하기 위한 조건문이 필요하다.
내부의 경우는 점이 삼각형 내부에 있을경우, 삼각형의 넓이와  삼각형 변 3개와 해당 점을 이은 삼각형 넓이의 합이 같은 점을 이용하여 구할 수 있고,
x,y축에 평행하지 않은 대각선 쪽의 바깥은 점과 대각선의 각 점의 거리를 비교하여 구할 수 있다.


In [112]:
class RTriObj extends ShapeObj { 
    int w; // 직각인 꼭지점으로부터 음수/양수로 왼쪽/오른쪽 방향으로의 w만큼 떨어진 꼭지점 나타냄
    int h; // 직각인 꼭지점으로부터 음수/양수로 아래쪽/위쪽 방향으로의 h만큼 떨어진 꼭지점 나타냄
    
    RTriObj(int w, int h, Point p, RGB c)  
    {
        super(p, c); 
        this.w = w;
        this.h = h;
    }
    @Override
    Point closest(Point p) 
    { 
        Point a = new Point (this.p.x+w,this.p.y); // x축 변화
        Point c = new Point (this.p.x,this.p.y+h); // y축 변화
        int maxX = Math.max(this.p.x, this.p.x+w),minX = Math.min(this.p.x, this.p.x+w); 
        int maxY = Math.max(this.p.y, this.p.y+h),minY = Math.min(this.p.y, this.p.y+h);
        int x,y; // 대각선 변 연산시 사용
        double py,px; //대각선 변 연산시 사용2(초기 반복문 시작위치)
        if(this.p.x>this.p.x+w && this.p.y>this.p.y+h) // 좌/상 방향으로 뻗은 삼각형(-,-) / 대각선모양
        {   //        c(+h)
            // a(+w)  b(this.p)
            if(p.x > maxX && (p.y < maxY && p.y > minY) ) // 오른쪽 바깥
                return new Point(maxX, p.y);
            else if( (p.x > minX && p.x < maxX) && p.y > maxY) // 아래쪽 바깥
                return new Point(p.x, maxY);
            else if( p.x >= maxX && p.y >= maxY) // 오른쪽 아래 모서리
                return this.p;
            else if( p.x >= maxX && p.y <= minY) // 오른쪽 위 모서리
                return c;
            else if( p.x <= minX && p.y >= maxY) // 왼쪽 아래 모서리
                return a;
            else if(Calc.EqArea(a,this.p,c,p)==true) // 내부
                return p;
            else 
            {   //내부가 아니면 결국 대각선이므로 대각선 검사용 세팅
                // / 방향은 아래에서 시작시 x+,y-
                x = c.x - a.x;
                y = c.y - a.y;
                px = a.x;
                py = a.y;
                return Calc.Shortlen(x,y,px,py,a,c,p);
            }
        }
        else if(this.p.x>this.p.x+w && this.p.y<this.p.y+h) // 좌/하 방향으로 뻗은 삼각형(-,+)
        {   // a(+w)  b(this.p)
            //        c(+h)
            if(p.x > maxX && (p.y > minY && p.y < maxY) ) // 오른쪽 바깥
                return new Point(maxX, p.y);
            else if( (p.x > minX && p.x < maxX) && p.y < minY) // 위쪽 바깥
                return new Point(p.x, minY);
            else if( p.x >= maxX && p.y >= maxY) // 오른쪽 아래 모서리
                return c;
            else if( p.x >= maxX && p.y <= minY) // 오른쪽 위 모서리
                return this.p;
            else if( p.x <= minX && p.y <= minY) // 왼쪽 위 모서리
                return a;
            else if(Calc.EqArea(a,this.p,c,p)==true) // 내부
                return p;
            else 
            { 
                //내부가 아니면 결국 대각선이므로 대각선 검사용 세팅
                // \ 방향은 왼쪽에서 시작시 x+,y+
                x = c.x - a.x;
                y = c.y - a.y;
                px = a.x;
                py = a.y;
                return Calc.Shortlen(x,y,px,py,a,c,p);
            }
        }
        else if(this.p.x<this.p.x+w && this.p.y<this.p.y+h) // 하/우 방향으로 뻗은 삼각형(+,+)
        {
            //   b(this.p) a(+w)
            //   c(+h)
            if(p.x < minX && (p.y > minY && p.y < maxY) ) // 왼쪽 바깥
                return new Point(minX, p.y);
            else if( (p.x > minX && p.x < maxX) && p.y < minY) // 위쪽 바깥
                return new Point(p.x, minY);
            else if( p.x <= minX && p.y >= maxY) // 왼쪽 아래 모서리
                return c;
            else if( p.x >= maxX && p.y <= minY) // 오른쪽 위 모서리
                return a;
            else if( p.x <= minX && p.y <= minY) // 왼쪽 위 모서리
                return this.p;
            else if(Calc.EqArea(a,this.p,c,p)==true) // 내부
                return p;
            else 
            { 
                //내부가 아니면 결국 대각선이므로 대각선 검사용 세팅
                // / 방향은 왼쪽에서 시작시 x+,y-
                x = a.x - c.x;
                y = a.y - c.y;
                px = c.x;
                py = c.y;
                return Calc.Shortlen(x,y,px,py,a,c,p);
            }
        }
        else  // 상/우 방향으로 뻗은 삼각형(+,-)
        {   //   c(+h)
            //   b(this.p) a(+w)
            if(p.x < minX && (p.y > minY && p.y < maxY) ) // 왼쪽 바깥
                return new Point(minX, p.y);
            else if( (p.x > minX && p.x < maxX) && p.y > maxY) // 아래쪽 바깥
                return new Point(p.x, maxY);
            else if( p.x <= minX && p.y >= maxY) // 왼쪽 아래 모서리
                return this.p;
            else if( p.x >= maxX && p.y <= maxY) // 오른쪽 아래 모서리
                return a;
            else if( p.x <= minX && p.y <= minY) // 왼쪽 위 모서리
                return c;
            else if(Calc.EqArea(a,this.p,c,p)==true) // 내부
                return p;
            else 
            { 
                //내부가 아니면 결국 대각선이므로 대각선 검사용 세팅
                // \ 방향은 왼쪽에서 시작시 x+,y+
                x = a.x - c.x;
                y = a.y - c.y;
                px = c.x;
                py = c.y;
                return Calc.Shortlen(x,y,px,py,a,c,p);
            }
        }    
    }
    @Override
    boolean collides(Point p) 
    { 
        Point p2 = closest(p);
        if(p2.x ==p.x && p2.y == p.y) //인자 p와 리턴값이 같다면 도형에 포함, 충돌
            return true;
        else
            return false;
    }
    @Override
    boolean collides(ShapeObj o) 
    { 
        Point a = new Point (this.p.x+w,this.p.y); // x축 변화
        Point c = new Point (this.p.x,this.p.y+h); // y축 변화
        Point po1 = closest(o.p); // 상대 중심과 내 도형의 위치(충돌-o.p,비충돌-내 도형의 경계) 저장 
        Point po2 = o.closest(p);
        if(o.collides(po1) || collides(po2)) // 내 도형의 중심이나 상대 도형의 경계가 내 도형과 충돌한다면 츙돌
            return true;
        else if (o.collides(this.p) || // if의 조건은 대각선/곡선의 경우 실수->정수 변환 과정에서 에러 가능, 따라서 각 모서리 충돌 검사
                 o.collides(a) ||
                 o.collides(c) )
            return true;
        else
            return false; 
    }
    @Override
    String toSVG() 
    { 
        return "<polygon points='"+p.x+","+p.y+" "+ (p.x+w)+","+p.y +" "+p.x+","+(p.y +h)+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }
}

com.twosigma.beaker.javash.bkr5a0ab621.RTriObj

## 두번째 설계
첫번째 설계와 같은 문제를 해결하지만 조금 다른 방식의 설계로 프로그램을 재작성해 보라.
* `Circle`, `Rectangle`, `RTriangle` 클래스를 작성하라 (1점)
* 첫번째에서 했던 같은 테스트 사례들로 모두 실행해 보라 (3점)

In [13]:
abstract class Shape { // 어떤 추상적인 모양 클래스
    RGB color; // 물건의 색깔
    Shape(RGB c) { color=c; }
    abstract String toSVG(Point self);
    abstract Point closest(Point self, Point p);
    abstract boolean collides(Point self, Point p);
    abstract boolean collides(Point self, Obj o);
}

class Obj {
    Point p;
    Shape s; // Shape이 Obj의 속성이다
    Obj(Point _p, Shape _s) { p=_p; s=_s; }
    Point closest(Point p) { return s.closest(this.p, p); } // 점 p와 가장 까까운 도형에 포함된 점 (소수점 이하는 적절히 처리해 정수 좌표로)
    boolean collides(Point p) { return s.collides(this.p, p); } // 점 p와 충돌하는가?
    boolean collides(Obj o) { return s.collides(this.p, o); } // o와 충돌하는가
    @Override
    public String toString() { return s.toSVG(this.p); }
}

com.twosigma.beaker.javash.bkr5a0ab621.Shape

In [35]:
class Circle extends Shape { // 원 모양 클래스
    int r;
    Circle(int r, RGB c) { super(c); this.r = r; }

    @Override
    Point closest(Point self, Point p) {
        double d = Calc.Length(self,p); // 두 점의 거리를 구한다.
        if(r > d) // 반지름이 길이보다 큰 경우, 점이 원 내부에 존재하므로 해당 점을 리턴
            return p; 
        else        // 반지름이 길이보다 작은경우, 직긱삼각형 비례를 이용하여 x,y 좌표를 구하여 리턴한다.
        {
            int dx = (int)Math.round( (p.x-self.x)*r/d ); 
            int dy = (int)Math.round( (p.y-self.y)*r/d );
            return new Point(self.x+dx, self.y+dy);
        }
    }

    @Override
    boolean collides(Point self, Point p) { // 원의 충돌검사의 경우, 좌표가 실수값 이기 때문에 두 점의 좌표 비교시 소수점문제로 인해 잘못된 결과 출력 가능성 존재
        double len = Calc.Length(self,p); // 두 점의 길이를 구한다.
        if( len <= r ) // 반지름이 길이보다 크거나 같은 경우, 점이 원 내부, 혹은 경계에 존재하므로 참
            return true;
        else           // 반지름이 길이보다 작은 경우, 점이 원 외부에 존재하므로 거짓
            return false;
    }
    @Override
    boolean collides(Point self, Obj o) {
        Point po = o.closest(self); // 상대 도형과 내 중심의 충돌정보를 저장할 포인트 변수 생성
        if(collides(self, po)) // po와 내 도형이 충돌한다면 참
            return true;
        else
            return false;
    }
    @Override
    String toSVG(Point self) {
        return "<circle cx='"+self.x+"' cy='"+self.y+"' r='"+r+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }
}

com.twosigma.beaker.javash.bkr5a0ab621.Circle

In [60]:
import com.twosigma.beakerx.widget.*;
HTML widget = new HTML();

Obj t1 = new Obj(new Point(150,150) , new Circle(70,new RGB(0,255,255))); // 70, 100 & 100 , 170

Point p1 =  new Point(80,80);

Circle c1  = new Circle(40 , new RGB(0,0,255));
Circle c2 = new Circle(5,  new RGB(255,0,0));

Obj o1 = new Obj(p1,c1);
Obj o2 = new Obj(t1.closest(p1),c2);
Obj o3 = new Obj(o1.closest(new Point(150,150)),c2);

String sgv = "";
sgv += "<p>보라색 원의 중점과 하늘색 원의 closest : "+o2.p+"</p>";
sgv +="<p>closest의 좌표와 보라색 원의 충돌검사  : "+o1.collides(o2.p)+"</p>";

sgv +=
    "<svg width='300' height='300' style='border:solid 1px'>"
    + t1 + o1 + o2 +
    "</svg>";

sgv += "<p>하늘색 원의 중점과 보라색 원의 closest : "+o3.p+"</p>";
sgv +="<p>closest의 좌표와 하늘색 원의 충돌검사  : "+t1.collides(o3.p)+"</p>";
sgv +=
    "<svg width='300' height='300' style='border:solid 1px'>"
    + t1 + o1 + o3 +
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");

return widget; 

In [94]:
class Rectangle extends Shape { 
    int w; // 너비 (양수)
    int h; // 높이 (양수)
    Rectangle(int w,int h, RGB c) {super(c); this.w = w; this.h = h;}
    
    @Override
    Point closest(Point self, Point p) 
    { 
        //case 9
        if(p.x < self.x && p.y < self.y) // 왼쪽 위 모서리
            return new Point(self.x,self.y);
        
        else if(p.x < self.x && p.y > self.y + this.h) // 왼쪽 아래 모서리
            return new Point(self.x,self.y+this.h);
        
        else if(p.x > self.x + this.w && p.y > self.y + this.h) // 오른쪽 아래 모서리
            return new Point(self.x+this.w,self.y+this.h);
        
        else if(p.x > self.x + this.w && p.y < self.y ) // 오른쪽 위 모서리
            return new Point(self.x + this.w, self.y);
        
        else if(p.x < self.x && (p.y >= self.y && p.y <= self.y + this.h) ) // 왼쪽 바깥
            return new Point(self.x,p.y);
        
        else if( (p.x >= self.x && p.x <= self.x + this.w) && p.y > self.y + this.h) // 아래쪽 바깥
            return new Point(p.x,self.y+this.h);
        
        else if( p.x > self.x + this.w && (p.y >= self.y && p.y <= self.y + this.h) ) // 오른쪽 바깥
            return new Point(self.x+this.w,p.y);
        
        else if(p.x >= self.x && p.x <= self.x + this.w && p.y < self.y) // 위쪽 바깥
            return new Point(p.x,self.y);
        else // 모두 아니면 사각형 안에 있으므로 해당 점 리턴 
            return p;
    }
    @Override
    boolean collides(Point self, Point p) {
        Point p2 = closest(self,p);
        if(p2.x ==p.x && p2.y == p.y) //인자 p와 리턴값이 같다면 도형에 포함, 충돌
            return true;
        else
            return false;
    }
    @Override
    boolean collides(Point self, Obj o) {
        Point po = closest(self,o.p); // 상대 중심과 내 도형의 위치(충돌-o.p,비충돌-내 도형의 경계) 저장 
        if(o.collides(po)==true) // 내 도형의 중심이나 상대 도형의 경계가 내 도형과 충돌한다면 츙돌
            return true;
        else if (o.collides(self) ||   // if의 조건은 대각선/곡선의 경우 실수->정수 변환 과정에서 에러 가능, 따라서 각 모서리 충돌 검사
                 o.collides(new Point(self.x+w,self.y  )) ||
                 o.collides(new Point(self.x,  self.y+h)) ||
                 o.collides(new Point(self.x+w,self.y+h)) ) 
            return true;
        else
            return false;
    }
    @Override
    String toSVG(Point self) 
    {
        return "<rect x='"+self.x+"' y='"+self.y+"' width='"+w+"'height='"+h+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }
}

com.twosigma.beaker.javash.bkr5a0ab621.Rectangle

In [84]:
import com.twosigma.beakerx.widget.*;
Obj t1 = new Obj(new Point(100,100) , new Rectangle(100,100,new RGB(0,255,255))); // 70, 100 & 100 , 170

Point p1 =  new Point(50,50);
Point p2 =  new Point(50,230);
Point p3 =  new Point(230,230);
Point p4 =  new Point(230,50);
Point p5 =  new Point(90,120);

Rectangle c1  = new Rectangle(30,30 , new RGB(255,0,0));
Rectangle c2  = new Rectangle(30,30  ,new RGB(255,100,0)); 
Rectangle c3  = new Rectangle(30,30  ,new RGB(255,255,0));
Rectangle c4  = new Rectangle(30,30  ,new RGB(0,255,0));
Rectangle c5  = new Rectangle(30,30  ,new RGB(255,0,255));

Circle c6  = new Circle(5,  new RGB(255,0,0));
Circle c7  = new Circle(5,  new RGB(255,100,0)); 
Circle c8  = new Circle(5,  new RGB(255,255,0));
Circle c9  = new Circle(5,  new RGB(0,255,0));
Circle c10 = new Circle(5,  new RGB(255,0,255)); 


Obj o1 = new Obj(t1.closest(p1),c6);
Obj o2 = new Obj(t1.closest(p2),c7);
Obj o3 = new Obj(t1.closest(p3),c8);
Obj o4 = new Obj(t1.closest(p4),c9);
Obj o5 = new Obj(t1.closest(p5),c10);
Obj o6 = new Obj(p1,c1);
Obj o7 = new Obj(p2,c2);
Obj o8 = new Obj(p3,c3);
Obj o9 = new Obj(p4,c4);
Obj o10 = new Obj(p5,c5);


System.out.println("하늘색 사각형과 빨주노초 사각형의 충돌검사 or : " +(o9.collides(t1) || o6.collides(t1) || o7.collides(t1) || o8.collides(t1)));
System.out.println("하늘색 사각형과 자주색 사각형의 충돌검사 : " +o10.collides(t1));

HTML widget = new HTML();
String sgv =
    "<svg width='300' height='300' style='border:solid 1px'>"
    + t1 + o1 + o2 + o3 + o4 + o5 + o6 + o7 + o8 + o9 + o10 +
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");


return widget; 

하늘색 사각형과 빨주노초 사각형의 충돌검사 or : false
하늘색 사각형과 자주색 사각형의 충돌검사 : true


In [96]:
class RTriangle extends Shape {
    int w; // 직각인 꼭지점으로부터 음수/양수로 왼쪽/오른쪽 방향으로의 w만큼 떨어진 꼭지점 나타냄
    int h; // 직각인 꼭지점으로부터 음수/양수로 아래쪽/위쪽 방향으로의 h만큼 떨어진 꼭지점 나타냄
    
    RTriangle(int w, int h, RGB c)  
    {
        super(c); // 부모 클래스 생성자 호출
        this.w = w;
        this.h = h;
    }
    @Override
    Point closest(Point self, Point p) 
    { // 점 p와 가장 까까운 도형에 포함된 점 (소수점 이하는 적절히 처리해 정수 좌표로)
        Point a = new Point (self.x+w,self.y); // x축 변화
        Point c = new Point (self.x,self.y+h); // y축 변화
        int maxX = Math.max(self.x, self.x+w),minX = Math.min(self.x, self.x+w); 
        int maxY = Math.max(self.y, self.y+h),minY = Math.min(self.y, self.y+h);
        int x,y; // 대각선 변 연산시 사용
        double py,px; //대각선 변 연산시 사용2(초기 반복문 시작위치)
        if(self.x>self.x+w && self.y>self.y+h) // 좌/상 방향으로 뻗은 삼각형(-,-) / 대각선모양
        {   //        c(+h)
            // a(+w)  b(self)
            if(p.x > maxX && (p.y < maxY && p.y > minY) ) // 오른쪽 바깥
                return new Point(maxX, p.y);
            else if( (p.x > minX && p.x < maxX) && p.y > maxY) // 아래쪽 바깥
                return new Point(p.x, maxY);
            else if( p.x >= maxX && p.y >= maxY) // 오른쪽 아래 모서리
                return self;
            else if( p.x >= maxX && p.y <= minY) // 오른쪽 위 모서리
                return c;
            else if( p.x <= minX && p.y >= maxY) // 왼쪽 아래 모서리
                return a;
            else if(Calc.EqArea(a,self,c,p)==true) // 내부
                return p;
            else 
            {
                // x+,y-
                x = c.x - a.x;
                y = c.y - a.y;
                px = a.x;
                py = a.y;
                return Calc.Shortlen(x,y,px,py,a,c,p);
            }
        }
        else if(self.x>self.x+w && self.y<self.y+h) // 좌/하 방향으로 뻗은 삼각형(-,+)
        {   // a(+w)  b(self)
            //        c(+h)
            if(p.x > maxX && (p.y > minY && p.y < maxY) ) // 오른쪽 바깥
                return new Point(maxX, p.y);
            else if( (p.x > minX && p.x < maxX) && p.y < minY) // 위쪽 바깥
                return new Point(p.x, minY);
            else if( p.x >= maxX && p.y >= maxY) // 오른쪽 아래 모서리
                return c;
            else if( p.x >= maxX && p.y <= minY) // 오른쪽 위 모서리
                return self;
            else if( p.x <= minX && p.y <= minY) // 왼쪽 위 모서리
                return a;
            else if(Calc.EqArea(a,self,c,p)==true) // 내부
                return p;
            else 
            { 
                // x+,y+
                x = c.x - a.x;
                y = c.y - a.y;
                px = a.x;
                py = a.y;
                return Calc.Shortlen(x,y,px,py,a,c,p);
            }
        }
        else if(self.x<self.x+w && self.y<self.y+h) // 하/우 방향으로 뻗은 삼각형(+,+)
        {
            //   b(self) a(+w)
            //   c(+h)
            if(p.x < minX && (p.y > minY && p.y < maxY) ) // 왼쪽 바깥
                return new Point(minX, p.y);
            else if( (p.x > minX && p.x < maxX) && p.y < minY) // 위쪽 바깥
                return new Point(p.x, minY);
            else if( p.x <= minX && p.y >= maxY) // 왼쪽 아래 모서리
                return c;
            else if( p.x >= maxX && p.y <= minY) // 오른쪽 위 모서리
                return a;
            else if( p.x <= minX && p.y <= minY) // 왼쪽 위 모서리
                return self;
            else if(Calc.EqArea(a,self,c,p)==true) // 내부
                return p;
            else 
            { 
                //내부가 아니면 결국 대각선이므로 대각선 검사용 세팅
                // / 방향은 왼쪽에서 시작시 x+,y-
                x = a.x - c.x;
                y = a.y - c.y;
                px = c.x;
                py = c.y;
                return Calc.Shortlen(x,y,px,py,a,c,p);
            }
        }
        else  // 상/우 방향으로 뻗은 삼각형(+,-)
        {   //   c(+h)
            //   b(self) a(+w)
            if(p.x < minX && (p.y > minY && p.y < maxY) ) // 왼쪽 바깥
                return new Point(minX, p.y);
            else if( (p.x > minX && p.x < maxX) && p.y > maxY) // 아래쪽 바깥
                return new Point(p.x, maxY);
            else if( p.x <= minX && p.y >= maxY) // 왼쪽 아래 모서리
                return self;
            else if( p.x >= maxX && p.y <= maxY) // 오른쪽 아래 모서리
                return a;
            else if( p.x <= minX && p.y <= minY) // 왼쪽 위 모서리
                return c;
            else if(Calc.EqArea(a,self,c,p)==true) // 내부
                return p;
            else 
            { 
                //내부가 아니면 결국 대각선이므로 대각선 검사용 세팅
                // \ 방향은 왼쪽에서 시작시 x+,y+
                x = a.x - c.x;
                y = a.y - c.y;
                px = c.x;
                py = c.y;
                return Calc.Shortlen(x,y,px,py,a,c,p);
            }
        }
    }
    @Override
    boolean collides(Point self, Point p) 
    {        
        Point p2 = closest(self,p);
        if(p2.x ==p.x && p2.y == p.y) //인자 p와 리턴값이 같다면 도형에 포함, 충돌
            return true;
        else
            return false;
    }
    @Override
    boolean collides(Point self, Obj o) 
    {
        Point a = new Point (self.x+w,self.y); // x축 변화
        Point c = new Point (self.x,self.y+h); // y축 변화
        
        Point po1 = closest(self,o.p); // 상대 중심과 내 도형의 위치(충돌-o.p,비충돌-내 도형의 경계) 저장 
        Point po2 = o.closest(self);
        if(o.collides(po1) || collides(self,po2)) // 내 도형의 중심이나 상대 도형의 경계가 내 도형과 충돌한다면 츙돌
            return true;
        else if (o.collides(self) || // if의 조건은 대각선/곡선의 경우 실수->정수 변환 과정에서 에러 가능, 따라서 각 모서리 충돌 검사
                 o.collides(a) ||
                 o.collides(c) )
            return true;

        else
            return false; 
    }    
        
    @Override
    String toSVG(Point self) 
    { 
        return "<polygon points='"+self.x+","+self.y+" "+ (self.x+w)+","+self.y +" "+self.x+","+(self.y +h)+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }

}

com.twosigma.beaker.javash.bkr5a0ab621.RTriangle

In [133]:
class TestData {
    public static String Vaild(){
        int count = 5;
        java.util.Random r = new java.util.Random();
        RGB red = new RGB(255,0,0);
        RGB blue = new RGB(0,0,255);

        String sgvs="";
        for(int i=0; i<count;i++)
        {
            Circle t1 = new Circle( r.nextInt(95)+5, red );
            Circle t2 = new Circle( r.nextInt(95)+5,blue );
            Obj o1 = new Obj(new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), t1);
            Obj o2 = new Obj(new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), t2);

            sgvs +=
            "<p>"+ i +" :" + o1.collides(o2)+"</p>"+ 
            "<svg width='300' height='300' style='border:solid 1px'>"
            + o1 + o2 +
            "</svg>"
            ;
        }
        for(int i=0; i<count;i++)
        {
            Rectangle t1 = new Rectangle( r.nextInt(190)+10, r.nextInt(190)+10,red );
            Rectangle t2 = new Rectangle( r.nextInt(190)+10, r.nextInt(190)+10,blue );
            Obj o1 = new Obj(new Point( r.nextInt(100)+50, r.nextInt(100)+50 ), t1);
            Obj o2 = new Obj(new Point( r.nextInt(100)+50, r.nextInt(100)+50 ), t2);

            sgvs +=
            "<p>"+ i +" :" + o1.collides(o2)+"</p>"+
            "<svg width='300' height='300' style='border:solid 1px'>"
            + o1 + o2 +
            "</svg>";
        }
        for(int i=0; i<count;i++)
        {
            RTriangle t1 = new RTriangle( r.nextInt(200)-100, r.nextInt(200)-100,red );
            RTriangle t2 = new RTriangle( r.nextInt(200)-100, r.nextInt(200)-100,blue );
            Obj o1 = new Obj(new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), t1);
            Obj o2 = new Obj(new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), t2);

            sgvs +=
            "<p>"+ i +" :" + o1.collides(o2)+"</p>" +
            "<svg width='300' height='300' style='border:solid 1px'>"
            + o1 + o2 +
            "</svg>";
           
        }
        for(int i=0; i<10;i++)
        {
            RTriangle t1 = new RTriangle( r.nextInt(200)-100, r.nextInt(200)-100,red );
            Rectangle t2 = new Rectangle( r.nextInt(190)+10, r.nextInt(190)+10,blue );
            Circle t3 = new Circle( r.nextInt(95)+5,new RGB(0,255,0) );
            Obj o1 = new Obj(new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), t1);
            Obj o2 = new Obj(new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), t2);
            Obj o3 = new Obj(new Point( r.nextInt(100)+100, r.nextInt(100)+100 ), t3);

            sgvs +=
            "<br/><br/><p>"+ i +" 삼각형<->사각형:" + o1.collides(o2)+","+o2.collides(o1)+
            "<p>    사각형<->원형:" + o2.collides(o3)+","+o3.collides(o2)+
            "<p>    원형<->삼각형:" + o3.collides(o1)+","+o1.collides(o3)+
            "</p>"+  
            "<svg width='300' height='300' style='border:solid 1px'>"
            + o1 + o2 + o3 + 
            "</svg>";

        }
         return sgvs;   
    }
}

com.twosigma.beaker.javash.bkr5a0ab621.TestData

In [134]:
import com.twosigma.beakerx.widget.*;
String s= TestData.Vaild();
HTML widget = new HTML();
widget.setValue("<div>"+s+"</div>");
return widget;

<!--
import com.twosigma.beakerx.widget.*;
Obj t1 = new Obj(new Point(150,150) , new Circle(70,new RGB(0,255,255))); // 70, 100 & 100 , 170

/*
Point p1 =  new Point(50,50);
Point p2 =  new Point(50,250);
Point p3 =  new Point(250,250);
Point p4 =  new Point(250,50);
Point p5 =  new Point(90,120);
*/

Point p1 =  new Point(80,80);
Point p2 =  new Point(80,170);
Point p3 =  new Point(170,170);
Point p4 =  new Point(170,80);
Point p5 =  new Point(90,120);
Circle c1  = new Circle(30 , new RGB(0,0,255));
Circle c2  = new Circle(30  ,new RGB(0,255,255)); 
Circle c3  = new Circle(30  ,new RGB(255,0,255));
Circle c4  = new Circle(30  ,new RGB(255,0,0));
Circle c5  = new Circle(30  ,new RGB(255,255,0));

Circle c6  = new Circle(5,  new RGB(0,0,255));
Circle c7  = new Circle(5,  new RGB(0,255,255)); 
Circle c8  = new Circle(5,  new RGB(255,0,255));
Circle c9  = new Circle(5,  new RGB(255,0,0));
Circle c10 = new Circle(5,  new RGB(255,255,0)); 



Obj o1 = new Obj(t1.closest(p1),c6);
Obj o2 = new Obj(t1.closest(p2),c7);
Obj o3 = new Obj(t1.closest(p3),c8);
Obj o4 = new Obj(t1.closest(p4),c9);
Obj o5 = new Obj(t1.closest(p5),c10);
Obj o6 = new Obj(p1,c1);
Obj o7 = new Obj(p2,c2);
Obj o8 = new Obj(p3,c3);
Obj o9 = new Obj(p4,c4);
Obj o10 = new Obj(p5,c5);




System.out.println(t1.collides(o6));
System.out.println(t1.collides(o7));
System.out.println(t1.collides(o8));
System.out.println(t1.collides(o9));
System.out.println(t1.collides(o10));

System.out.println(o6.collides(t1));
System.out.println(o7.collides(t1));
System.out.println(o8.collides(t1));
System.out.println(o9.collides(t1));
System.out.println(o10.collides(t1));

HTML widget = new HTML();
String sgv =
    "<svg width='300' height='300' style='border:solid 1px'>"
    + t1 + o1 + o2 + o3 + o4 + o5 + o6 + o7 + o8 + o9 + o10 +
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");


return widget; -->

<!--import com.twosigma.beakerx.widget.*;

RGB red = new RGB(255,0,0);
RGB blue = new RGB(0,0,255);

CircObj c1 = new CircObj(30, new Point( 150, 150), red);
CircObj c2 = new CircObj(10, new Point(50,50), blue);
CircObj c3 = new CircObj(10, new Point(50,250), red);
CircObj c4 = new CircObj(10, new Point(250,250), blue);
CircObj c5 = new CircObj(10, new Point(250,50), red);

Point p1 = c1.closest(c2.p);
CircObj c6 = new CircObj(5, p1, new RGB(0,0,0));

Point p2 = c1.closest(c3.p);
CircObj c7 = new CircObj(5, p2, new RGB(255,0,0)); // R

Point p3 = c1.closest(c4.p);
CircObj c8 = new CircObj(5, p3, new RGB(0,255,0)); //G

Point p4 = c1.closest(c5.p);
CircObj c9 = new CircObj(5, p4, new RGB(0,0,255)); //B



HTML widget = new HTML();
String sgv =
    "<svg width='300' height='300' style='border:solid 1px'>"
    + c1 + c2 + c3 + c4 + c5 +c6 + c7 + c8 + c9 +
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");
return widget; // 그림에 나타나는 것과 맞는지 시각적으로 확인해 볼 수 있도록 toSVG를 정의한 것이다
-->

<!--
    @Override
    Point closest(Point p) // 점 p와 가장 까까운 도형에 포함된 점 
    { 
        double length = Calc.Length(this.p,p); // 중심점과 점간의 거리
        if(r > length) // 반지름이 길이보다 큰 경우, 점이 원 내부에 존재하므로 해당 점을 리턴
            return p;
        else           // 반지름이 길이보다 작은 경우, 점이 외부에 위치하므로 원 경계의 좌표를 리턴
        {
            double angle   = Math.toRadians(Math.atan2(difY,difX) * (180.0/Math.PI)-90);
            // 라디안값을 구한 후 각도로 바꾸어 보정치 부여, 다시 라디안값으로 변환 (atan 사용시 사분면 전체 각도 불가)
            
            double borderx = this.p.x + r * Math.sin(angle); 
            double bordery = this.p.y - r * Math.cos(angle); // 디스플레이는 y축 방향이 반대이므로
            return new Point((int)Math.round(borderx),(int)Math.round(bordery)); // round 함수로 반올림하여 정수로
        }
    }
    -->

<!--import com.twosigma.beakerx.widget.*;

RTriObj r1 = new RTriObj(100, -100, new Point( 100,200),new RGB(0,255,255));
RTriObj r2 = new RTriObj(40, -70, new Point( 150,205),new RGB(255,0,0));
System.out.println("하늘/빨강"+ r1.collides(r2) ); // 이런 결과가

HTML widget = new HTML();
String sgv =
    "<svg width='300' height='300' style='border:solid 1px'>"
    + r1 + r2 +
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");


return widget; // 그림에 나타나는 것과 맞는지 시각적으로 확인해 볼 수 있도록 toSVG를 정의한 것이다-->

<!--import com.twosigma.beakerx.widget.*;
RTriObj t1 = new RTriObj(-30, 70, new Point(100,100),new RGB(0,255,255)); // 70, 100 & 100 , 170

Point p1 =  new Point(50,50);
Point p2 =  new Point(50,250);
Point p3 =  new Point(250,250);
Point p4 =  new Point(250,50);
Point p5 =  new Point(90,120);
CircObj c1 = new CircObj(5, t1.closest(p1), new RGB(0,0,255));
CircObj c2 = new CircObj(5, t1.closest(p2), new RGB(0,255,255)); //
CircObj c3 = new CircObj(5, t1.closest(p3), new RGB(255,0,255));
CircObj c4 = new CircObj(5, t1.closest(p4), new RGB(255,0,0));
CircObj c9 = new CircObj(5, t1.closest(p5), new RGB(255,255,0));
CircObj c5 = new CircObj(5, p1, new RGB(0,0,255));
CircObj c6 = new CircObj(5, p2, new RGB(0,255,255)); //
CircObj c7 = new CircObj(5, p3, new RGB(255,0,255));
CircObj c8 = new CircObj(5, p4, new RGB(255,0,0));
CircObj c10 = new CircObj(5, p5, new RGB(255,255,0));
System.out.println(t1.collides(t1.closest(p1)));
System.out.println(t1.collides(t1.closest(p2)));
System.out.println(t1.collides(t1.closest(p3)));
System.out.println(t1.collides(t1.closest(p4)));
System.out.println(t1.collides(t1.closest(p5)));
HTML widget = new HTML();
String sgv =
    "<svg width='300' height='300' style='border:solid 1px'>"
    + t1 + c1  + c2 + c3 + c4 + c5 + c6  + c7 +c8+c9+c10+
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");


return widget; // 그림에 나타나는 것과 맞는지 시각적으로 확인해 볼 수 있도록 toSVG를 정의한 것이다 */ -->

<!--
import com.twosigma.beakerx.widget.*;
int x1 = 30,x2 = 40;
int y1 = 40,y2 = 60;
CircObj c1 = new CircObj(20, new Point(x1,y1), new RGB(255,0,0));
CircObj c2 = new CircObj(10, new Point(x2,y2), new RGB(0,255,0));
CircObj c3 = new CircObj(2, c1.closest(c2.p), new RGB(0,0,0));

HTML widget = new HTML();
String sgv =
    "<svg width='100' height='100' style='border:solid 1px'>"
    + c1 +c2 + c3+
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");

System.out.println(c1.collides(c2));

return widget; 
-->